In [2]:
# ============================================================================
# ANÁLISIS DE RIESGO DE TORRES DE TRANSMISIÓN
# Combina: Amenaza estática + Precipitación actual = Nivel de alerta
# ============================================================================

# %% [markdown]
# ## 1. Importar librerías

# %%
import pandas as pd
import numpy as np
import rasterio
import requests
import plotly.graph_objects as go
import plotly.express as px
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

print("✅ Librerías importadas")

# %% [markdown]
# ## 2. Cargar ubicaciones de torres

# %%
# Cargar CSV de torres
ruta_torres = "../data/03_external/ubicacion_torres.csv"
torres_df = pd.read_csv(ruta_torres)

print("=" * 70)
print("📍 TORRES CARGADAS")
print("=" * 70)
print(f"\nTotal de torres: {len(torres_df)}")
print(f"\nPrimeras torres:")
display(torres_df.head())

# Validar datos
print(f"\n✅ Rango de coordenadas:")
print(f"   Latitud:  {torres_df['Latitud'].min():.4f} a {torres_df['Latitud'].max():.4f}")
print(f"   Longitud: {torres_df['Longitud'].min():.4f} a {torres_df['Longitud'].max():.4f}")

# %% [markdown]
# ## 2.5 Verificar cobertura del raster

# %%
print("🔍 Verificando cobertura del raster para las torres...\n")

ruta_tiff_original = "../data/01_raw/AmeMM_100k.tif"

with rasterio.open(ruta_tiff_original) as src:
    bounds = src.bounds
    
    print(f"📊 Límites del raster:")
    print(f"   Norte: {bounds.top:.4f}°")
    print(f"   Sur:   {bounds.bottom:.4f}°")
    print(f"   Este:  {bounds.right:.4f}°")
    print(f"   Oeste: {bounds.left:.4f}°")
    
    print(f"\n📍 Límites de las torres:")
    print(f"   Norte: {torres_df['Latitud'].max():.4f}°")
    print(f"   Sur:   {torres_df['Latitud'].min():.4f}°")
    print(f"   Este:  {torres_df['Longitud'].max():.4f}°")
    print(f"   Oeste: {torres_df['Longitud'].min():.4f}°")
    
    # Verificar si las torres están dentro del raster
    torres_dentro = 0
    torres_fuera = []
    
    for idx, torre in torres_df.iterrows():
        lat, lon = torre['Latitud'], torre['Longitud']
        
        if (bounds.bottom <= lat <= bounds.top and 
            bounds.left <= lon <= bounds.right):
            torres_dentro += 1
        else:
            torres_fuera.append(torre['ID_Torre'])
    
    print(f"\n✅ Torres dentro del raster: {torres_dentro}/{len(torres_df)}")
    
    if torres_fuera:
        print(f"⚠️  Torres FUERA del raster: {len(torres_fuera)}")
        print(f"   {', '.join(torres_fuera)}")
        print("\n   💡 Estas torres no tendrán datos de amenaza")
    else:
        print("✅ Todas las torres están dentro del área del raster")

# %% [markdown]
# ## 3. Extraer nivel de amenaza estática para cada torre
# 
# Lee directamente del TIFF original y clasifica

# %%
# Ruta al archivo ORIGINAL
ruta_tiff_original = "../data/01_raw/AmeMM_100k.tif"

print("🔍 Extrayendo valores de amenaza para cada torre...\n")

# Crear lista de coordenadas (lon, lat)
coords = [(lon, lat) for lon, lat in zip(torres_df['Longitud'], torres_df['Latitud'])]

# Extraer valores del raster ORIGINAL
with rasterio.open(ruta_tiff_original) as src:
    print(f"📊 Raster: {src.width}x{src.height} píxeles")
    print(f"🌍 CRS: {src.crs}")
    print(f"📍 Límites: {src.bounds}\n")
    
    # Muestrear el raster en las coordenadas de las torres
    valores_amenaza_continuos = []
    
    for i, (lon, lat) in enumerate(coords):
        torre_id = torres_df.iloc[i]['ID_Torre']
        try:
            # Intentar muestrear
            valor = list(src.sample([(lon, lat)]))[0][0]
            
            # Verificar si es NoData
            if src.nodata is not None and valor == src.nodata:
                print(f"   ⚠️  {torre_id}: Sin datos en esta ubicación")
                valores_amenaza_continuos.append(np.nan)
            else:
                print(f"   ✅ {torre_id}: Valor = {valor:.2f}")
                valores_amenaza_continuos.append(valor)
        except Exception as e:
            print(f"   ❌ {torre_id}: Error - {e}")
            valores_amenaza_continuos.append(np.nan)

# Agregar valores continuos al dataframe
torres_df['Amenaza_Valor_Continuo'] = valores_amenaza_continuos

# Calcular umbrales si hay datos válidos
valores_validos = [v for v in valores_amenaza_continuos if not np.isnan(v)]

if len(valores_validos) > 0:
    # Calcular percentiles para clasificación
    umbral_bajo = np.percentile(valores_validos, 33)
    umbral_alto = np.percentile(valores_validos, 66)
    
    print(f"\n🎯 Umbrales calculados:")
    print(f"   🟢 Baja:  ≤ {umbral_bajo:.2f}")
    print(f"   🟡 Media: > {umbral_bajo:.2f} y ≤ {umbral_alto:.2f}")
    print(f"   🔴 Alta:  > {umbral_alto:.2f}")
    
    # Clasificar valores
    def clasificar_amenaza(valor):
        if np.isnan(valor):
            return 0, 'Sin Datos'
        elif valor <= umbral_bajo:
            return 1, 'Baja'
        elif valor <= umbral_alto:
            return 2, 'Media'
        else:
            return 3, 'Alta'
    
    torres_df[['Amenaza_Valor', 'Amenaza_Nivel']] = torres_df['Amenaza_Valor_Continuo'].apply(
        lambda x: pd.Series(clasificar_amenaza(x))
    )
else:
    print("\n❌ No se encontraron datos válidos para ninguna torre")
    print("   Verifica que las coordenadas estén dentro del área del raster")
    torres_df['Amenaza_Valor'] = 0
    torres_df['Amenaza_Nivel'] = 'Sin Datos'

# Mostrar resultados
print("\n✅ Valores de amenaza extraídos:\n")

# Filtrar solo columnas relevantes
columnas_mostrar = ['ID_Torre', 'Nombre', 'Latitud', 'Longitud', 
                    'Amenaza_Valor_Continuo', 'Amenaza_Nivel']
display(torres_df[columnas_mostrar])

# Resumen
print(f"\n📊 Distribución de amenaza en torres:")
distribucion = torres_df['Amenaza_Nivel'].value_counts()
for nivel in ['Baja', 'Media', 'Alta', 'Sin Datos']:
    count = distribucion.get(nivel, 0)
    if count > 0:
        emoji = {'Baja': '🟢', 'Media': '🟡', 'Alta': '🔴', 'Sin Datos': '⚪'}
        print(f"   {emoji[nivel]} {nivel}: {count} torres")

# Advertencia si hay torres sin datos
torres_sin_datos = (torres_df['Amenaza_Nivel'] == 'Sin Datos').sum()
if torres_sin_datos > 0:
    print(f"\n⚠️  ADVERTENCIA: {torres_sin_datos} torre(s) sin datos")
    print("   Posibles causas:")
    print("   • Las coordenadas están fuera del área cubierta por el raster")
    print("   • Error en el sistema de coordenadas (CRS)")
    print("   • Valores NoData en esas ubicaciones del raster")
    print("\n   Solución: Verifica las coordenadas o usa un raster que cubra toda el área")

# %% [markdown]
# ## 4. Obtener datos de precipitación de Open-Meteo
# 
# Obtenemos datos de las últimas 72 horas para calcular acumulados

# %%
def obtener_lluvia_torre(lat, lon, dias_historico=3):
    """
    Obtiene datos de precipitación de Open-Meteo para una ubicación.
    
    Args:
        lat: Latitud
        lon: Longitud
        dias_historico: Días hacia atrás para obtener datos
    
    Returns:
        DataFrame con datos de precipitación horaria
    """
    # Calcular fechas
    end_date = datetime.now()
    start_date = end_date - timedelta(days=dias_historico)
    
    # Construir URL
    url = (
        "https://api.open-meteo.com/v1/forecast?"
        f"latitude={lat}&longitude={lon}"
        "&hourly=precipitation"
        f"&start_date={start_date.strftime('%Y-%m-%d')}"
        f"&end_date={end_date.strftime('%Y-%m-%d')}"
    )
    
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        data = response.json()
        
        if 'hourly' in data and 'time' in data['hourly']:
            df = pd.DataFrame(data['hourly'])
            df['time'] = pd.to_datetime(df['time'])
            return df
        else:
            return None
    
    except Exception as e:
        print(f"   ⚠️  Error obteniendo datos: {e}")
        return None


print("🌧️  Obteniendo datos de precipitación para cada torre...")
print("   (Esto puede tomar 30-60 segundos)\n")

# Lista para almacenar resultados
datos_lluvia = []

for idx, torre in torres_df.iterrows():
    print(f"   📡 {torre['ID_Torre']}: {torre['Nombre']}...", end=" ")
    
    df_lluvia = obtener_lluvia_torre(torre['Latitud'], torre['Longitud'], dias_historico=3)
    
    if df_lluvia is not None and not df_lluvia.empty:
        # Calcular acumulado de 72 horas
        lluvia_72h = df_lluvia['precipitation'].sum()
        
        # Últimas 24 horas
        lluvia_24h = df_lluvia.tail(24)['precipitation'].sum()
        
        datos_lluvia.append({
            'ID_Torre': torre['ID_Torre'],
            'Lluvia_72h': lluvia_72h,
            'Lluvia_24h': lluvia_24h
        })
        
        print(f"✅ {lluvia_72h:.1f}mm (72h)")
    else:
        datos_lluvia.append({
            'ID_Torre': torre['ID_Torre'],
            'Lluvia_72h': 0,
            'Lluvia_24h': 0
        })
        print("⚠️  Sin datos")

# Convertir a DataFrame y combinar
df_lluvia_torres = pd.DataFrame(datos_lluvia)
torres_df = torres_df.merge(df_lluvia_torres, on='ID_Torre', how='left')

print("\n✅ Datos de precipitación obtenidos")

# %% [markdown]
# ## 5. Aplicar matriz de riesgo y calcular nivel de alerta
# 
# Matriz de decisión:
# - **Amenaza ALTA**: Alerta AMARILLA si lluvia > 60mm, ROJA si > 100mm
# - **Amenaza MEDIA**: Alerta AMARILLA si lluvia > 120mm
# - **Amenaza BAJA**: Sin alertas automáticas

# %%
def calcular_nivel_alerta(row):
    """
    Calcula el nivel de alerta basado en amenaza estática y lluvia acumulada.
    """
    amenaza = row['Amenaza_Nivel']
    lluvia_72h = row['Lluvia_72h']
    
    if amenaza == 'Alta':
        if lluvia_72h > 100:
            return 'ROJA', '🔴'
        elif lluvia_72h > 60:
            return 'AMARILLA', '🟡'
    elif amenaza == 'Media':
        if lluvia_72h > 120:
            return 'AMARILLA', '🟡'
    
    return 'VERDE', '🟢'

# Aplicar función
torres_df[['Nivel_Alerta', 'Emoji_Alerta']] = torres_df.apply(
    calcular_nivel_alerta, axis=1, result_type='expand'
)

# Calcular prioridad (para ordenar)
prioridad_map = {'ROJA': 3, 'AMARILLA': 2, 'VERDE': 1}
torres_df['Prioridad'] = torres_df['Nivel_Alerta'].map(prioridad_map)

# Ordenar por prioridad (más urgente primero)
torres_df = torres_df.sort_values('Prioridad', ascending=False)

print("=" * 70)
print("🚨 NIVEL DE ALERTA CALCULADO PARA CADA TORRE")
print("=" * 70)
print()

# Mostrar tabla completa con colores
display(torres_df[[
    'ID_Torre', 'Nombre', 'Amenaza_Nivel', 
    'Lluvia_72h', 'Lluvia_24h', 'Emoji_Alerta', 'Nivel_Alerta'
]].style.background_gradient(subset=['Lluvia_72h'], cmap='Blues'))

# %% [markdown]
# ## 6. Resumen ejecutivo

# %%
print("=" * 70)
print("📊 RESUMEN EJECUTIVO - SITUACIÓN ACTUAL")
print("=" * 70)
print(f"\n🕐 Fecha y hora: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"📍 Total de torres monitoreadas: {len(torres_df)}")

print("\n🚨 ALERTAS ACTIVAS:")
n_rojas = (torres_df['Nivel_Alerta'] == 'ROJA').sum()
n_amarillas = (torres_df['Nivel_Alerta'] == 'AMARILLA').sum()
n_verdes = (torres_df['Nivel_Alerta'] == 'VERDE').sum()

print(f"   🔴 ROJAS (Críticas):    {n_rojas} torres")
print(f"   🟡 AMARILLAS (Precaución): {n_amarillas} torres")
print(f"   🟢 VERDES (Normal):     {n_verdes} torres")

if n_rojas > 0:
    print(f"\n⚠️  ATENCIÓN: {n_rojas} torre(s) en alerta ROJA requieren atención inmediata:")
    torres_rojas = torres_df[torres_df['Nivel_Alerta'] == 'ROJA']
    for _, torre in torres_rojas.iterrows():
        print(f"      • {torre['ID_Torre']}: {torre['Nombre']} - {torre['Lluvia_72h']:.1f}mm en 72h")

if n_amarillas > 0:
    print(f"\n⚠️  {n_amarillas} torre(s) en alerta AMARILLA requieren monitoreo:")
    torres_amarillas = torres_df[torres_df['Nivel_Alerta'] == 'AMARILLA']
    for _, torre in torres_amarillas.iterrows():
        print(f"      • {torre['ID_Torre']}: {torre['Nombre']} - {torre['Lluvia_72h']:.1f}mm en 72h")

if n_rojas == 0 and n_amarillas == 0:
    print("\n✅ Todas las torres están en estado normal (VERDE)")

# %% [markdown]
# ## 7. Visualización: Mapa interactivo de torres

# %%
# Crear mapa con Plotly
fig = go.Figure()

# Colores por nivel de alerta
color_map = {
    'VERDE': 'green',
    'AMARILLA': 'orange',
    'ROJA': 'red'
}

# Tamaños por nivel de alerta
size_map = {
    'VERDE': 10,
    'AMARILLA': 15,
    'ROJA': 20
}

for nivel in ['VERDE', 'AMARILLA', 'ROJA']:
    torres_nivel = torres_df[torres_df['Nivel_Alerta'] == nivel]
    
    if not torres_nivel.empty:
        fig.add_trace(go.Scattermapbox(
            lat=torres_nivel['Latitud'],
            lon=torres_nivel['Longitud'],
            mode='markers',
            marker=dict(
                size=torres_nivel['Nivel_Alerta'].map(size_map),
                color=color_map[nivel]
            ),
            text=torres_nivel.apply(
                lambda x: f"<b>{x['ID_Torre']}: {x['Nombre']}</b><br>" +
                         f"Amenaza: {x['Amenaza_Nivel']}<br>" +
                         f"Lluvia 72h: {x['Lluvia_72h']:.1f}mm<br>" +
                         f"Alerta: {x['Nivel_Alerta']}",
                axis=1
            ),
            hoverinfo='text',
            name=f'Alerta {nivel}'
        ))

# Configurar mapa
fig.update_layout(
    mapbox=dict(
        style='open-street-map',
        center=dict(
            lat=torres_df['Latitud'].mean(),
            lon=torres_df['Longitud'].mean()
        ),
        zoom=8
    ),
    title=dict(
        text=f'Mapa de Torres con Nivel de Alerta - {datetime.now().strftime("%Y-%m-%d %H:%M")}',
        font=dict(size=18, color='black')
    ),
    height=600,
    margin=dict(l=0, r=0, t=40, b=0)
)

fig.show()

print("🗺️  Mapa interactivo generado")
print("   💡 Haz zoom y haz clic en los marcadores para ver detalles")

# %% [markdown]
# ## 8. Gráfico: Lluvia acumulada vs. Umbrales de alerta

# %%
fig = go.Figure()

# Barras de lluvia acumulada coloreadas por nivel de alerta
colores_barras = torres_df['Nivel_Alerta'].map(color_map)

fig.add_trace(go.Bar(
    x=torres_df['ID_Torre'],
    y=torres_df['Lluvia_72h'],
    marker=dict(color=colores_barras),
    text=torres_df['Lluvia_72h'].round(1),
    textposition='outside',
    name='Lluvia 72h'
))

# Líneas de umbral según amenaza
# Umbral crítico general (Alta amenaza)
fig.add_hline(y=100, line_dash="dash", line_color="red", 
              annotation_text="Umbral ROJO (Alta: 100mm)", 
              annotation_position="right")
fig.add_hline(y=60, line_dash="dash", line_color="orange", 
              annotation_text="Umbral AMARILLO (Alta: 60mm)", 
              annotation_position="right")

fig.update_layout(
    title=dict(
        text='Precipitación Acumulada (72h) por Torre',
        font=dict(size=16, color='black')
    ),
    xaxis_title='Torre',
    yaxis_title='Precipitación (mm)',
    height=500,
    showlegend=False,
    hovermode='x unified'
)

fig.show()

print("📊 Gráfico de barras generado")

# %% [markdown]
# ## 9. Gráfico: Matriz de riesgo (Amenaza vs Lluvia)

# %%
fig = px.scatter(
    torres_df,
    x='Lluvia_72h',
    y='Amenaza_Valor',
    color='Nivel_Alerta',
    color_discrete_map=color_map,
    size='Prioridad',
    size_max=20,
    hover_data=['ID_Torre', 'Nombre', 'Amenaza_Nivel'],
    labels={
        'Lluvia_72h': 'Precipitación Acumulada 72h (mm)',
        'Amenaza_Valor': 'Nivel de Amenaza Estática'
    },
    title='Matriz de Riesgo: Amenaza Estática vs Precipitación'
)

# Agregar zonas de riesgo
fig.add_vrect(x0=0, x1=60, fillcolor="green", opacity=0.1, line_width=0)
fig.add_vrect(x0=60, x1=100, fillcolor="yellow", opacity=0.1, line_width=0)
fig.add_vrect(x0=100, x1=torres_df['Lluvia_72h'].max()*1.1, 
              fillcolor="red", opacity=0.1, line_width=0)

# Ajustar eje Y para mostrar categorías
fig.update_yaxes(
    tickmode='array',
    tickvals=[1, 2, 3],
    ticktext=['Baja', 'Media', 'Alta']
)

fig.update_layout(height=500)
fig.show()

print("📊 Matriz de riesgo generada")

# %% [markdown]
# ## 10. Guardar resultados

# %%
# Guardar tabla completa
ruta_salida = "../data/02_processed/torres_con_amenaza_y_alerta.csv"
torres_df.to_csv(ruta_salida, index=False)

print("=" * 70)
print("💾 RESULTADOS GUARDADOS")
print("=" * 70)
print(f"\n📁 Archivo guardado en:")
print(f"   {ruta_salida}")

print(f"\n📊 Columnas incluidas:")
for col in torres_df.columns:
    print(f"   • {col}")

print("\n✅ ¡Análisis completado exitosamente!")
print("\n🚀 PRÓXIMOS PASOS:")
print("   1. Revisar torres en alerta ROJA o AMARILLA")
print("   2. Implementar acciones preventivas según protocolo")
print("   3. Ejecutar este notebook periódicamente para monitoreo continuo")
print("   4. Crear dashboard de Streamlit para visualización en tiempo real")

# %% [markdown]
# ## 11. Reporte en texto para compartir

# %%
# Generar reporte de texto
reporte = f"""
{'='*70}
REPORTE DE ALERTAS - SISTEMA DE MONITOREO DE TORRES
{'='*70}

Fecha y Hora: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
Total de Torres: {len(torres_df)}

RESUMEN DE ALERTAS:
-------------------
🔴 ROJAS (Críticas):      {n_rojas} torre(s)
🟡 AMARILLAS (Precaución): {n_amarillas} torre(s)
🟢 VERDES (Normal):       {n_verdes} torre(s)

"""

if n_rojas > 0:
    reporte += "\nTORRES EN ALERTA ROJA (ACCIÓN INMEDIATA REQUERIDA):\n"
    reporte += "-" * 70 + "\n"
    for _, torre in torres_rojas.iterrows():
        reporte += f"""
Torre: {torre['ID_Torre']} - {torre['Nombre']}
   • Ubicación: {torre['Latitud']:.4f}°, {torre['Longitud']:.4f}°
   • Amenaza Estática: {torre['Amenaza_Nivel']}
   • Precipitación 72h: {torre['Lluvia_72h']:.1f} mm
   • Precipitación 24h: {torre['Lluvia_24h']:.1f} mm
   • NIVEL DE ALERTA: 🔴 ROJA
"""

if n_amarillas > 0:
    reporte += "\nTORRES EN ALERTA AMARILLA (MONITOREO REQUERIDO):\n"
    reporte += "-" * 70 + "\n"
    for _, torre in torres_amarillas.iterrows():
        reporte += f"""
Torre: {torre['ID_Torre']} - {torre['Nombre']}
   • Ubicación: {torre['Latitud']:.4f}°, {torre['Longitud']:.4f}°
   • Amenaza Estática: {torre['Amenaza_Nivel']}
   • Precipitación 72h: {torre['Lluvia_72h']:.1f} mm
   • Precipitación 24h: {torre['Lluvia_24h']:.1f} mm
   • NIVEL DE ALERTA: 🟡 AMARILLA
"""

reporte += f"""
{'='*70}
CRITERIOS DE ALERTA:

Amenaza ALTA:
   • Alerta AMARILLA: Precipitación > 60mm en 72h
   • Alerta ROJA: Precipitación > 100mm en 72h

Amenaza MEDIA:
   • Alerta AMARILLA: Precipitación > 120mm en 72h

Amenaza BAJA:
   • Sin alertas automáticas
{'='*70}
"""

# Guardar reporte
with open("../data/02_processed/reporte_alertas.txt", "w", encoding='utf-8') as f:
    f.write(reporte)

print(reporte)
print("\n💾 Reporte guardado en: ../data/02_processed/reporte_alertas.txt")

✅ Librerías importadas
📍 TORRES CARGADAS

Total de torres: 7

Primeras torres:


,ID_Torre,Nombre,Latitud,Longitud
0,TORRE_001,Torre 1,6.642631,-71.814700
1,TORRE_002,Torre 2,6.858107,-71.902591
2,TORRE_003,Torre 3,6.841745,-71.391727
3,TORRE_004,Torre 4,6.356094,-70.825931
4,TORRE_005,Torre 5,6.710830,-70.666629



✅ Rango de coordenadas:
   Latitud:  6.2141 a 6.8581
   Longitud: -71.9465 a -70.6666
🔍 Verificando cobertura del raster para las torres...

📊 Límites del raster:
   Norte: 1871234.6213°
   Sur:   23958.2629°
   Este:  1803779.5814°
   Oeste: 451571.1883°

📍 Límites de las torres:
   Norte: 6.8581°
   Sur:   6.2141°
   Este:  -70.6666°
   Oeste: -71.9465°

✅ Torres dentro del raster: 0/7
⚠️  Torres FUERA del raster: 7
   TORRE_001, TORRE_002, TORRE_003, TORRE_004, TORRE_005, TORRE_006, TORRE_007

   💡 Estas torres no tendrán datos de amenaza
🔍 Extrayendo valores de amenaza para cada torre...

📊 Raster: 43811x59851 píxeles
🌍 CRS: PROJCS["MAGNA_Transverse_Mercator",GEOGCS["MAGNA-SIRGAS",DATUM["Marco_Geocentrico_Nacional_de_Referencia",SPHEROID["GRS 1980",6378137,298.257222101004,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6686"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4686"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitud

,ID_Torre,Nombre,Latitud,Longitud,Amenaza_Valor_Continuo,Amenaza_Nivel
0,TORRE_001,Torre 1,6.642631,-71.814700,NaN,Sin Datos
1,TORRE_002,Torre 2,6.858107,-71.902591,NaN,Sin Datos
2,TORRE_003,Torre 3,6.841745,-71.391727,NaN,Sin Datos
3,TORRE_004,Torre 4,6.356094,-70.825931,NaN,Sin Datos
4,TORRE_005,Torre 5,6.710830,-70.666629,NaN,Sin Datos
5,TORRE_006,Torre 6,6.634446,-71.314822,NaN,Sin Datos
6,TORRE_007,Torre 7,6.214130,-71.946536,NaN,Sin Datos



📊 Distribución de amenaza en torres:
   ⚪ Sin Datos: 7 torres

⚠️  ADVERTENCIA: 7 torre(s) sin datos
   Posibles causas:
   • Las coordenadas están fuera del área cubierta por el raster
   • Error en el sistema de coordenadas (CRS)
   • Valores NoData en esas ubicaciones del raster

   Solución: Verifica las coordenadas o usa un raster que cubra toda el área
🌧️  Obteniendo datos de precipitación para cada torre...
   (Esto puede tomar 30-60 segundos)

   📡 TORRE_001: Torre 1... ✅ 32.0mm (72h)
   📡 TORRE_002: Torre 2... ✅ 46.3mm (72h)
   📡 TORRE_003: Torre 3... ✅ 34.3mm (72h)
   📡 TORRE_004: Torre 4... ✅ 31.5mm (72h)
   📡 TORRE_005: Torre 5... ✅ 27.7mm (72h)
   📡 TORRE_006: Torre 6... ✅ 35.8mm (72h)
   📡 TORRE_007: Torre 7... ✅ 40.6mm (72h)

✅ Datos de precipitación obtenidos
🚨 NIVEL DE ALERTA CALCULADO PARA CADA TORRE



,ID_Torre,Nombre,Amenaza_Nivel,Lluvia_72h,Lluvia_24h,Emoji_Alerta,Nivel_Alerta
0,TORRE_001,Torre 1,Sin Datos,32.000000,8.600000,🟢,VERDE
1,TORRE_002,Torre 2,Sin Datos,46.300000,24.000000,🟢,VERDE
2,TORRE_003,Torre 3,Sin Datos,34.300000,9.000000,🟢,VERDE
3,TORRE_004,Torre 4,Sin Datos,31.500000,10.400000,🟢,VERDE
4,TORRE_005,Torre 5,Sin Datos,27.700000,9.900000,🟢,VERDE
5,TORRE_006,Torre 6,Sin Datos,35.800000,7.500000,🟢,VERDE
6,TORRE_007,Torre 7,Sin Datos,40.600000,10.300000,🟢,VERDE


📊 RESUMEN EJECUTIVO - SITUACIÓN ACTUAL

🕐 Fecha y hora: 2025-10-11 20:25:16
📍 Total de torres monitoreadas: 7

🚨 ALERTAS ACTIVAS:
   🔴 ROJAS (Críticas):    0 torres
   🟡 AMARILLAS (Precaución): 0 torres
   🟢 VERDES (Normal):     7 torres

✅ Todas las torres están en estado normal (VERDE)


🗺️  Mapa interactivo generado
   💡 Haz zoom y haz clic en los marcadores para ver detalles


📊 Gráfico de barras generado


📊 Matriz de riesgo generada
💾 RESULTADOS GUARDADOS

📁 Archivo guardado en:
   ../data/02_processed/torres_con_amenaza_y_alerta.csv

📊 Columnas incluidas:
   • ID_Torre
   • Nombre
   • Latitud
   • Longitud
   • Amenaza_Valor_Continuo
   • Amenaza_Valor
   • Amenaza_Nivel
   • Lluvia_72h
   • Lluvia_24h
   • Nivel_Alerta
   • Emoji_Alerta
   • Prioridad

✅ ¡Análisis completado exitosamente!

🚀 PRÓXIMOS PASOS:
   1. Revisar torres en alerta ROJA o AMARILLA
   2. Implementar acciones preventivas según protocolo
   3. Ejecutar este notebook periódicamente para monitoreo continuo
   4. Crear dashboard de Streamlit para visualización en tiempo real

REPORTE DE ALERTAS - SISTEMA DE MONITOREO DE TORRES

Fecha y Hora: 2025-10-11 20:25:17
Total de Torres: 7

RESUMEN DE ALERTAS:
-------------------
🔴 ROJAS (Críticas):      0 torre(s)
🟡 AMARILLAS (Precaución): 0 torre(s)
🟢 VERDES (Normal):       7 torre(s)


CRITERIOS DE ALERTA:

Amenaza ALTA:
   • Alerta AMARILLA: Precipitación > 60mm en 72h
   